In [ ]:
from ffnn import FFNN

import plotly.offline as plotly
import plotly.graph_objs as go

from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.manifold import TSNE

In [ ]:
plotly.init_notebook_mode(connected=True)

# Data

In [ ]:
# Load in data as numpy arrays
train_in = np.load('data/train_in.npy')
train_labels = np.load('data/train_labels.npy')
test_in = np.load('data/test_in.npy')
test_labels = np.load('data/test_labels.npy')

In [ ]:
# Lets take a look at one of our images!
Image.fromarray((train_in[1] * 255).astype('uint8').reshape(28, 28), mode='L')

# Visualizations

In [ ]:
tsne = TSNE()
visualization = tsne.fit_transform(train_in[:5000])

In [ ]:
plotly.iplot([
    go.Scatter(
        x=visualization[:, 0],
        y=visualization[:, 1],
        mode='markers',
        marker=dict(
            size=3,
            # color = train_labels[:5000].argmax(axis=1),
            # colorscale='Rainbow'
        )
    )
])

# Define Model

In [ ]:
# Define the neural network structure
# Here we have just one hidden layer with 2048 nodes
network = FFNN([784, 2048, 10], post_function=tf.nn.softmax)

In [ ]:
# Train the model using gradient descent
def cross_entropy_with_softmax(model_output, true_output):
    return tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=true_output, logits=model_output))

network.train(
    train_in,
    train_labels,
    epochs=1100,
    batch_size=100,
    lc_interval=300,
    loss_func=cross_entropy_with_softmax
)

In [ ]:
# Lets check our error rate on our train set
train_outputs = network.evaluate(train_in)
print("Correct predictions in train set: {}".format(
    np.sum(train_outputs.argmax(axis=1) == train_labels.argmax(axis=1)) / float(train_labels.shape[0])
))

In [ ]:
# And check that the predictions match the image
i = 5
print("Prediction: {}".format(train_outputs[i].argmax()))
Image.fromarray((train_in[i] * 255).astype('uint8').reshape(28, 28), mode='L')

# Evaluate Model

In [ ]:
test_outputs = network.evaluate(test_in)
print("Correct predictions in test_set: {}".format(
    np.sum(test_outputs.argmax(axis=1) == test_labels.argmax(axis=1)) / float(test_labels.shape[0])
))

In [ ]:
i = 5
print("Prediction: {}".format(test_outputs[i].argmax()))
Image.fromarray((test_in[i] * 255).astype('uint8').reshape(28, 28), mode='L')